In [1]:
%load_ext autoreload
import json
import datetime
import pandas as pd
import re
import cProfile, pstats

### Load data

In [2]:
%autoreload 2
from fairiskdata import FAIRiskDataset

%time dataset = FAIRiskDataset.load()


CPU times: user 16.2 s, sys: 331 ms, total: 16.5 s
Wall time: 17 s


### Low-level access to dataset (dictionary)

In [3]:
print(dataset.get().keys())
print(dataset.get()['Qatar'].keys())
print(dataset.get()['Portugal'].keys())

dict_keys(['Afghanistan', 'Albania', 'Algeria', 'Andorra', 'Angola', 'Anguilla', 'Antigua and Barbuda', 'Argentina', 'Armenia', 'Australia', 'Austria', 'Azerbaijan', 'Bahamas', 'Bahrain', 'Bangladesh', 'Barbados', 'Belarus', 'Belgium', 'Belize', 'Benin', 'Bermuda', 'Bhutan', 'Bolivia', 'Bosnia and Herzegovina', 'Botswana', 'Brazil', 'Brunei Darussalam', 'Bulgaria', 'Burkina Faso', 'Burundi', 'Cambodia', 'Cameroon', 'Canada', 'Cabo Verde', 'Cayman Islands', 'Central African Republic', 'Chad', 'Chile', 'China', 'Colombia', 'Comoros', 'Congo Republic', 'Costa Rica', "Cote d'Ivoire", 'Croatia', 'Cuba', 'Cyprus', 'Czech Republic', 'DR Congo', 'Denmark', 'Djibouti', 'Dominica', 'Dominican Republic', 'Ecuador', 'Egypt', 'El Salvador', 'Equatorial Guinea', 'Eritrea', 'Estonia', 'Eswatini', 'Ethiopia', 'Faeroe Islands', 'Falkland Islands', 'Fiji', 'Finland', 'France', 'Gabon', 'Gambia', 'Georgia', 'Germany', 'Ghana', 'Gibraltar', 'Greece', 'Greenland', 'Grenada', 'Guatemala', 'Guernsey', 'Guine

In [4]:
for key, value in dataset.get()['Portugal']['DEMOGRAPHIC'].items():
    print(f"{key} >> {value.keys()}\n")
    print(f"{value['FREQUENCY'] if 'FREQUENCY' in value else ''}")

85 years or over_Females >> dict_keys(['ATTR_NAME', 'SOURCE', 'UNIT', 'FREQUENCY', 'SERIES_TYPE', 'VALUE'])

YEARLY
85 years or over_Males >> dict_keys(['ATTR_NAME', 'SOURCE', 'UNIT', 'FREQUENCY', 'SERIES_TYPE', 'VALUE'])

YEARLY
85 years or over_Total >> dict_keys(['ATTR_NAME', 'SOURCE', 'UNIT', 'FREQUENCY', 'SERIES_TYPE', 'VALUE'])

YEARLY
From 10 to 14 years_Females >> dict_keys(['ATTR_NAME', 'SOURCE', 'UNIT', 'FREQUENCY', 'SERIES_TYPE', 'VALUE'])

YEARLY
From 10 to 14 years_Males >> dict_keys(['ATTR_NAME', 'SOURCE', 'UNIT', 'FREQUENCY', 'SERIES_TYPE', 'VALUE'])

YEARLY
From 10 to 14 years_Total >> dict_keys(['ATTR_NAME', 'SOURCE', 'UNIT', 'FREQUENCY', 'SERIES_TYPE', 'VALUE'])

YEARLY
From 15 to 19 years_Females >> dict_keys(['ATTR_NAME', 'SOURCE', 'UNIT', 'FREQUENCY', 'SERIES_TYPE', 'VALUE'])

YEARLY
From 15 to 19 years_Males >> dict_keys(['ATTR_NAME', 'SOURCE', 'UNIT', 'FREQUENCY', 'SERIES_TYPE', 'VALUE'])

YEARLY
From 15 to 19 years_Total >> dict_keys(['ATTR_NAME', 'SOURCE', 'UNI

In [5]:
print(dataset.get()['Portugal']['COVID']['total_cases'].keys())
print(dataset.get()['Portugal']['COVID']['total_cases']['VALUE'].keys())

dict_keys(['ATTR_NAME', 'SOURCE', 'UNIT', 'VALUE', 'FREQUENCY', 'SERIES_TYPE'])
Index(['2020-02-02', '2020-02-09', '2020-02-16', '2020-02-23', '2020-03-01',
       '2020-03-02', '2020-03-03', '2020-03-04', '2020-03-05', '2020-03-06',
       ...
       '2021-02-14', '2021-02-15', '2021-02-16', '2021-02-17', '2021-02-18',
       '2021-02-19', '2021-02-20', '2021-02-21', '2021-02-22', '2021-02-23'],
      dtype='object', length=364)


### Filtering examples

In [6]:
%autoreload 2

dataset = FAIRiskDataset.load()

# new api
%time print(dataset.filter_countries(["Portugal", "Spain"]).get().keys())
%time print(dataset.filter_countries(["Portugal", "Spain"]).get()["Portugal"].keys())
dataset = FAIRiskDataset.load()
%time print(dataset.filter_countries(["Portugal", "Spain"]).filter_categories("COVID").get()["Portugal"].keys())
%time print(dataset.filter_countries(["Portugal", "Spain"]).filter_categories("COVID").get()["Portugal"]["COVID"].keys())
dataset = FAIRiskDataset.load()
%time print(dataset.filter_attributes([("COVID", "total_deaths"), ("COVID", "new_cases")]).get()["Portugal"]["COVID"].keys())
%time print(dataset.filter_attributes([("COVID", "total_deaths"), ("COVID", "new_cases")]).get()["Portugal"]["COVID"]["total_deaths"].keys())


et - WARNING - Eritrea no longer has DEMOGRAPHIC, INDICATORS, SCORES data.
2021-03-12 10:00:31,421 - fairisk_dataset - WARNING - Estonia no longer has DEMOGRAPHIC, INDICATORS, SCORES, MORTALITY, MOBILITY data.
2021-03-12 10:00:31,421 - fairisk_dataset - WARNING - Eswatini no longer has DEMOGRAPHIC, INDICATORS, SCORES data.
2021-03-12 10:00:31,422 - fairisk_dataset - WARNING - Ethiopia no longer has DEMOGRAPHIC, INDICATORS, SCORES, MOBILITY data.
2021-03-12 10:00:31,422 - fairisk_dataset - WARNING - Faeroe Islands no longer has DEMOGRAPHIC, INDICATORS, SCORES, MOBILITY data.
2021-03-12 10:00:31,423 - fairisk_dataset - WARNING - Falkland Islands no longer has DEMOGRAPHIC, INDICATORS, SCORES data.
2021-03-12 10:00:31,423 - fairisk_dataset - WARNING - Fiji no longer has DEMOGRAPHIC, INDICATORS, SCORES, MOBILITY data.
2021-03-12 10:00:31,424 - fairisk_dataset - WARNING - Finland no longer has DEMOGRAPHIC, INDICATORS, SCORES, MORTALITY, MOBILITY data.
2021-03-12 10:00:31,424 - fairisk_datase

In [7]:
%autoreload 2

from fairiskdata.utils.time_parsers import safe_date_parse

# between two years
dataset = FAIRiskDataset.load()
%time print(dataset.filter_time_interval(pd.Interval(pd.Timestamp("2020"), pd.Timestamp("2021"), closed='neither')).get()["Portugal"]["COVID"]["total_deaths"]["VALUE"].keys())
# year
dataset = FAIRiskDataset.load()
%time print(dataset.filter_time_interval(pd.Period("2014")).get()["Portugal"]["COVID"]["total_deaths"]["VALUE"].keys())
# week
dataset = FAIRiskDataset.load()
%time print(dataset.filter_time_interval(safe_date_parse('2020W01')).get()[\"Portugal\"][\"COVID\"][\"total_deaths\"][\"VALUE\"].keys())

2021-03-12 10:02:02,850 - fairisk_dataset - WARNING - Andorra no longer has DEMOGRAPHIC data.
2021-03-12 10:02:02,852 - fairisk_dataset - WARNING - Armenia no longer has DEMOGRAPHIC data.
2021-03-12 10:02:02,853 - fairisk_dataset - WARNING - Azerbaijan no longer has DEMOGRAPHIC data.
2021-03-12 10:02:02,854 - fairisk_dataset - WARNING - Belarus no longer has DEMOGRAPHIC data.
2021-03-12 10:02:02,856 - fairisk_dataset - WARNING - Bosnia and Herzegovina no longer has DEMOGRAPHIC data.
2021-03-12 10:02:02,863 - fairisk_dataset - WARNING - France no longer has DEMOGRAPHIC data.
2021-03-12 10:02:02,865 - fairisk_dataset - WARNING - Georgia no longer has DEMOGRAPHIC data.
2021-03-12 10:02:02,870 - fairisk_dataset - WARNING - Kosovo no longer has DEMOGRAPHIC data.
2021-03-12 10:02:02,875 - fairisk_dataset - WARNING - Moldova no longer has DEMOGRAPHIC data.
2021-03-12 10:02:02,876 - fairisk_dataset - WARNING - Monaco no longer has DEMOGRAPHIC data.
2021-03-12 10:02:02,881 - fairisk_dataset - W

KeyError: 'COVID'

In [16]:
%autoreload 2

# new api
dataset = FAIRiskDataset.load()
%time print(dataset.filter_age_group(age_group=(10,25)).get()["Portugal"]["DEMOGRAPHIC"].keys())
%time print(dataset.filter_age_group(age_group=(10,25)).get()["Portugal"]["MORTALITY"].keys())

dict_keys(['From 10 to 14 years_Females', 'From 10 to 14 years_Males', 'From 10 to 14 years_Total', 'From 15 to 19 years_Females', 'From 15 to 19 years_Males', 'From 15 to 19 years_Total', 'From 20 to 24 years_Females', 'From 20 to 24 years_Males', 'From 20 to 24 years_Total', 'From 25 to 29 years_Females', 'From 25 to 29 years_Males', 'From 25 to 29 years_Total', 'Total_Females', 'Total_Males', 'Total_Total'])
CPU times: user 193 ms, sys: 2.13 ms, total: 195 ms
Wall time: 196 ms
dict_keys(['From 10 to 14 years_Females', 'From 10 to 14 years_Males', 'From 10 to 14 years_Total', 'From 15 to 19 years_Females', 'From 15 to 19 years_Males', 'From 15 to 19 years_Total', 'From 20 to 24 years_Females', 'From 20 to 24 years_Males', 'From 20 to 24 years_Total', 'From 25 to 29 years_Females', 'From 25 to 29 years_Males', 'From 25 to 29 years_Total', 'Total_Females', 'Total_Males', 'Total_Total'])
CPU times: user 183 ms, sys: 1.74 ms, total: 185 ms
Wall time: 186 ms


In [14]:
%autoreload 2

dataset = FAIRiskDataset.load()
%time print(dataset.filter_countries_with_missing_values_on_attributes([('MORTALITY', 'D0_14_b')]).get().keys())

dataset = FAIRiskDataset.load()
%time print(dataset.filter_countries_with_missing_values_on_attributes([('MORTALITY', 'D0_14_b')]).get()['United States']['MORTALITY']['D0_14_b']["VALUE"]["2014W01"])
dataset = FAIRiskDataset.load()
%time print(dataset.filter_countries_with_missing_values_on_attributes([('MORTALITY', 'D0_14_b')]).filter_time_interval(pd.Period("2014")).get()['United States']['MORTALITY']['D0_14_b'])

dict_keys(['Australia', 'Austria', 'Belgium', 'Bulgaria', 'Canada', 'Chile', 'Croatia', 'Czech Republic', 'Denmark', 'Estonia', 'Finland', 'France', 'Germany', 'Greece', 'Hungary', 'Iceland', 'Israel', 'Italy', 'Latvia', 'Lithuania', 'Luxembourg', 'Netherlands', 'New Zealand', 'Norway', 'Poland', 'Portugal', 'Russia', 'Slovakia', 'Slovenia', 'South Korea', 'Spain', 'Sweden', 'Switzerland', 'Taiwan', 'United Kingdom', 'United States'])
CPU times: user 43.3 ms, sys: 4.29 ms, total: 47.6 ms
Wall time: 47.7 ms
605.1883640945359
CPU times: user 46.8 ms, sys: 3.46 ms, total: 50.2 ms
Wall time: 50.6 ms
{'ATTR_NAME': 'D0_14_b', 'SOURCE': 'Human Mortality Database', 'UNIT': 'Number', 'FREQUENCY': 'WEEKLY', 'VALUE': 2014W01    605.188364
2014W02    627.275531
2014W03    612.256257
2014W04    624.625071
2014W05    580.450738
2014W06    621.974611
2014W07    624.625071
2014W08    608.722311
2014W09    620.207637
2014W10    623.741584
2014W11    574.266331
2014W12    611.372771
2014W13    600.77093

In [15]:
%autoreload 2

# new api
dataset = FAIRiskDataset.load()
#profiler = cProfile.Profile()
#profiler.enable()
%time print(dataset.filter_countries_missing_value_attributes_below(18).get().keys())
#profiler.disable()


dict_keys(['Bulgaria', 'Italy', 'Slovenia', 'Spain'])
CPU times: user 864 ms, sys: 9.98 ms, total: 874 ms
Wall time: 874 ms


In [8]:
%autoreload 2

# new api
dataset = FAIRiskDataset.load()
#profiler = cProfile.Profile()
#profiler.enable()
print(f"Number of attributeds before: {len(dataset.get_attributes())}")
%time dataset.filter_attributes_with_countries_nan_below(70)
print(f"Number of attributeds after: {len(dataset.get_attributes())}")
#profiler.disable()

Number of attributeds before: 69449
CPU times: user 4.72 s, sys: 38.1 ms, total: 4.76 s
Wall time: 4.79 s
Number of attributeds after: 44343


In [16]:
%autoreload 2
dataset = FAIRiskDataset.load()

%time print(dataset.filter_time_interval(pd.Period("2013")).get()['United States']['MORTALITY']['D0_14_b']["VALUE"].keys())

Index(['2013W01', '2013W02', '2013W03', '2013W04', '2013W05', '2013W06',
       '2013W07', '2013W08', '2013W09', '2013W10', '2013W11', '2013W12',
       '2013W13', '2013W14', '2013W15', '2013W16', '2013W17', '2013W18',
       '2013W19', '2013W20', '2013W21', '2013W22', '2013W23', '2013W24',
       '2013W25', '2013W26', '2013W27', '2013W28', '2013W29', '2013W30',
       '2013W31', '2013W32', '2013W33', '2013W34', '2013W35', '2013W36',
       '2013W37', '2013W38', '2013W39', '2013W40', '2013W41', '2013W42',
       '2013W43', '2013W44', '2013W45', '2013W46', '2013W47', '2013W48',
       '2013W49', '2013W50', '2013W51', '2013W52', '2014W01'],
      dtype='object')
CPU times: user 55.4 s, sys: 339 ms, total: 55.8 s
Wall time: 56.1 s


### Resampling

In [13]:
%%time
%autoreload 2
dataset.filter_countries('Portugal').resample('MONTHLY')
dataset.get()['Portugal']['MORTALITY']['Total_Total']['VALUE'].keys()

CPU times: user 29.6 s, sys: 320 ms, total: 29.9 s
Wall time: 30.2 s


Index(['01-2000', '02-2000', '03-2000', '04-2000', '05-2000', '06-2000',
       '07-2000', '08-2000', '09-2000', '10-2000',
       ...
       '02-2020', '03-2020', '04-2020', '05-2020', '06-2020', '07-2020',
       '08-2020', '09-2020', '10-2020', '11-2020'],
      dtype='object', length=251)

### Exporting dataframes

In [4]:
%%time
%autoreload 2
e = dataset.export(type='parameters')
print(len(e.columns))
e

100
CPU times: user 7.37 ms, sys: 1.49 ms, total: 8.85 ms
Wall time: 15.7 ms


,country,DEMOGRAPHIC:population,DEMOGRAPHIC:median_age,DEMOGRAPHIC:aged_65_older,DEMOGRAPHIC:aged_70_older,INDICATORS:gdp_per_capita,INDICATORS:extreme_poverty,INDICATORS:cardiovasc_death_rate,INDICATORS:diabetes_prevalence,INDICATORS:female_smokers,...,SCORES:MPI,SCORES:GII,SCORES:GINI,SCORES:AtC,SCORES:IHR09,SCORES:Ptip,SCORES:HFA,SCORES:POL.GE.EST,SCORES:CPI,SCORES:SDGIHR
0,Portugal,10196707.0,46.2,21.502,14.924,27936.896,0.5,127.842,9.85,16.3,...,None,0.080794,35.5,3.851772,80.0,3.210778,3.95,1.208909,62.0,77.0


In [4]:
%%time
%autoreload 2
e = dataset.export(type='all')
print(len(e.columns))
e

10
CPU times: user 5.54 s, sys: 546 ms, total: 6.08 s
Wall time: 6.19 s


,country,category,attribute,ATTR_NAME,SOURCE,UNIT,FREQUENCY,SERIES_TYPE,key,value
0,Afghanistan,COVID,total_cases,total_cases,Data on COVID-19 (coronavirus) by Our World in...,Number,DAILY,TOTAL,2020-02-24,1
1,Afghanistan,COVID,total_cases,total_cases,Data on COVID-19 (coronavirus) by Our World in...,Number,DAILY,TOTAL,2020-02-25,1
2,Afghanistan,COVID,total_cases,total_cases,Data on COVID-19 (coronavirus) by Our World in...,Number,DAILY,TOTAL,2020-02-26,1
3,Afghanistan,COVID,total_cases,total_cases,Data on COVID-19 (coronavirus) by Our World in...,Number,DAILY,TOTAL,2020-02-27,1
4,Afghanistan,COVID,total_cases,total_cases,Data on COVID-19 (coronavirus) by Our World in...,Number,DAILY,TOTAL,2020-02-28,1
...,...,...,...,...,...,...,...,...,...,...
1882996,United States Virgin Islands,MOBILITY,all_day_ratio_single_tile_users_std,all_day_ratio_single_tile_users_std,Movement Range Maps from Facebook,Relation to baseline (full_february - DAY_OF_W...,DAILY,CURRENT,2021-01-22,NaN
1882997,United States Virgin Islands,MOBILITY,all_day_ratio_single_tile_users_std,all_day_ratio_single_tile_users_std,Movement Range Maps from Facebook,Relation to baseline (full_february - DAY_OF_W...,DAILY,CURRENT,2021-01-23,NaN
1882998,United States Virgin Islands,MOBILITY,all_day_ratio_single_tile_users_std,all_day_ratio_single_tile_users_std,Movement Range Maps from Facebook,Relation to baseline (full_february - DAY_OF_W...,DAILY,CURRENT,2021-01-24,NaN
1882999,United States Virgin Islands,MOBILITY,all_day_ratio_single_tile_users_std,all_day_ratio_single_tile_users_std,Movement Range Maps from Facebook,Relation to baseline (full_february - DAY_OF_W...,DAILY,CURRENT,2021-01-25,NaN


In [7]:
%%time
%autoreload 2
e = dataset.export(type='timeseries')
print(len(e.columns))
e


152
CPU times: user 57.4 s, sys: 1.98 s, total: 59.4 s
Wall time: 60 s


In [10]:
e.parsed_timestamp.unique()

array(['1989-12-25T00:00:00.000000000', '1990-01-01T00:00:00.000000000',
       '1990-01-08T00:00:00.000000000', ...,
       '2020-12-28T00:00:00.000000000', '2021-01-04T00:00:00.000000000',
                                 'NaT'], dtype='datetime64[ns]')